$$
\begin{array}{rl1}
  & \text{let } x_{t,e,w,s} \\
  & \text t \text{ = type of employee} \\
  & \text e \text{ = employee} \\ 
  & \text w \text{ = hour of work} \\
  & \text s \text{ = the hour that start} \\
 \end{array}
$$

$$
\begin{array}{rll}
 & \text{Min total daily wages}\\
 \text{s.t.} 
 & \text{Each worker only works for between their min&max work hours} \\
 & \text{Each worker only works at their available hours} \\
 & \text{Each worker only works for one shift (if called for duty)} \\
 & \text{Minimum number of staff needed for work is satisfied at each hour} \\
 & \text{non-negativity constraints}\\
\end{array}
$$

### Set up Gurobi environment

In [ ]:
from gurobipy import *
m = Model()

### Add variables, where type i_j stands for min hour i to max hour j

In [ ]:
employees = ["SMITH", "JOHNSON", "WILLIAMS", "JONES", "BROWN", "DAVIS", "MILLER","WILSON",
            "MOORE", "TAYLOR", "ANDERSON", "THOMAS", "JACKSON", "WHITE", "HARRIS", "MARTIN",
            "THOMPSON", "GARCIA", "MARTINEZ", "ROBINSON"]

types = ["2_3", "2_4", "2_5", "2_6", "6_8"]

hours = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", 
          "17", "18", "19", "20", "21", "22", "23"]

#for convenience when adding constraint
available_constrs = ["SMITH", "MILLER", "ANDERSON", "JACKSON", "THOMPSON"]

In [ ]:
employee_types = {"2_3":{"ANDERSON", "MARTIN"},
         "2_4":{"THOMAS", "JACKSON", "GARCIA", "MARTINEZ"},
         "2_5":{"THOMPSON", "ROBINSON"},
         "2_6":{"WHITE", "HARRIS"},
         "6_8":{"SMITH", "JOHNSON", "WILLIAMS", "JONES", "BROWN", "DAVIS", "MILLER", "WILSON", "MOORE", "TAYLOR"}
        }

# i.e for a 2_3 type, the worker can work either 2 hour or 3 hour is hired
workhour_types = {"2_3":{"2", "3"},
         "2_4":{"2", "3", "4"},
         "2_5":{"2", "3", "4", "5"},
         "2_6":{"2", "3", "4", "5", "6"},
         "6_8":{"6", "7", "8"}
           }

# Start_hours: the time that the worker start to work
start_hours =  ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", 
          "17", "18", "19", "20", "21", "22", "23"]


#available hours for some employees
available_hours = {"SMITH":{"6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"},
                   "MILLER":{"6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18"},
                   "ANDERSON":{"0", "1", "2", "3", "4", "5", "6", "18", "19", "20", "21", "22", "23"},
                   "JACKSON":{"8", "9", "10", "11", "12", "13", "14", "15", "16"},
                   "THOMPSON":{"12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"}
                  }
# the type of employees that have constraint on their available hours
available_types = {"SMITH":"6_8",
                   "MILLER":"6_8",
                   "ANDERSON":"2_3",
                   "JACKSON":"2_4",
                   "THOMPSON":"2_5"}

In [ ]:
wage_contribution = {"SMITH":30, "JOHNSON":50, "WILLIAMS":30, "JONES":30, "BROWN":40, "DAVIS":50, "MILLER":45,"WILSON":30,
            "MOORE":35, "TAYLOR":40, "ANDERSON":60, "THOMAS":40, "JACKSON":60, "WHITE":55, "HARRIS":45, "MARTIN":40,
            "THOMPSON":50, "GARCIA":50, "MARTINEZ":40, "ROBINSON":50}

### Employer's demand at each given hour

In [ ]:
employer_demand = {"0":1, "1":1, "2":2, "3":3, "4":6, "5":6, "6":7, "7":8, "8":9,
               "9":8, "10":8, "11":8, "12":7, "13":6, "14":6, "15":5, "16":5,
               "17":4, "18":4, "19":3, "20":2, "21":2, "22":2, "23":2
              }

### Load variables into the Gurobi Model
$$
\begin{array}{rll}
& \text{hire_employee_hours[t][e,w,s]: } x_{t,e,w,s} = 1\\
& \text{if the type t worker e start working time s for w hours, otherwise 0}\\
& \text{hire_employee[e]: } y_{e} = 1\\
& \text{if the employee is hired, otherwise 0}\\
\end{array}
$$

In [ ]:
hire_employee_hours = {}
for t in types:
    hire_employee_hours[t] = m.addVars(employee_types[t], workhour_types[t], start_hours)
    

hire_employee = m.addVars(employees, name = "hire_employee")

### Set up variable type (binary decisions)

In [ ]:
for t in types:
    for employee in employee_types[t]:
        for workhour in workhour_types[t]:
            for start_hour in start_hours:
                hire_employee_hours[t][employee,workhour,start_hour].vtype = GRB.BINARY
            
for employee in employees:
    hire_employee[employee].vtype = GRB.BINARY

In [ ]:
m.update()

### Add constraints (one shift per worker if called for duty)

In [ ]:
for t in types:
    m.addConstrs((quicksum(hire_employee_hours[t][e, w, s] for s in start_hours 
                       for w in workhour_types[t]) == hire_employee[e] for e in employee_types[t]), name = "one_shift_constraints") 

### Add constraints (available hours)

In [ ]:
for e in available_constrs:
    t = available_types[e] # the corresponding type of the employee, for the brevity of notation
    m.addConstr(quicksum(hire_employee_hours[t][e,w,s] 
                          for s in available_hours[e] for w in workhour_types[t] 
                          if (str((int(s) + int(w) - 1) % 25) in available_hours[e])) == hire_employee[e])


### Add constraints (minimum number of worker needed)

In [ ]:
m.addConstrs((quicksum(hire_employee_hours[t][e, w, s]for t in types for e in employee_types[t]
                       for s in start_hours for w in workhour_types[t] if ((( int(s) < int(h) )and ((int(s) + int(w)) > int(h)) ) or (int(s) == int(h)) or ( (int(s) > int(h)) and ( (int(s) + int(w)) > 24 + int(h)) ) )
                      ) >= employer_demand[h] for h in hours), name = "demand_constraints")


In [ ]:
m.update() 

### Set objective function

In [ ]:
obj = quicksum( (int(w) *  hire_employee_hours[t][e, w, s] * wage_contribution[e]) for t in types for s in start_hours 
                       for w in workhour_types[t] for e in employee_types[t])

### Tell Gurobi we want to minimize this objective function

In [ ]:
m.setObjective(obj, GRB.MINIMIZE)

### Solve the model

In [ ]:
m.optimize()

In [ ]:
m.write("file.lp")

### Set up matlab environment for plotting results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### Optimal decision output

In [ ]:
for t in types:
    for e in employee_types[t]:
        for w in workhour_types[t]:
            for s in start_hours:
                if(hire_employee_hours[t][e,w,s].X == 1):
                    print("hire employee " + e + " at time " + s + " for " + w + "hours")

### Employee shift schedule

In [ ]:
employees_schedule = {}

for t in types:
    for e in employee_types[t]:
        # clear the temporary list
        employee_schedule = [0]*24
        if(hire_employee[e].X == 1): # this employee is hired
            for w in workhour_types[t]:
                for s in start_hours:
                    if(hire_employee_hours[t][e,w,s].X == 1):
                        for i in range(0, int(w)):
                            # use a temporary list to store the schedule for current worker
                            employee_schedule[(int(s) + i) % 24] += 1 
                    # use a dictionary of list to store the schedule for each worker
                    employees_schedule[e] = employee_schedule.copy()
        else: 
            employees_schedule[e] = employee_schedule.copy()

### Hourly staff count (demand and actual)

In [ ]:
# actual employee working at each hour
actual_employee_working = [0] * 24

for t in types:
    for e in employee_types[t]:
        for w in workhour_types[t]:
            for s in start_hours:
                if(hire_employee_hours[t][e,w,s].X == 1):
                    for i in range(0, int(w)):
                        actual_employee_working[(int(s) + i) % 24] += 1


# demand employee working at each hour
demand_employee_working = list(map(int, employer_demand.values()))

### Plot the results

In [ ]:
# convert the dictionary into a matrix for plotting
employee_matrix = np.array([employees_schedule[i] for i in employees])
plt.imshow(employee_matrix);
#titles and labels
plt.xlabel('Time of day')
plt.ylabel('Employee Name')
plt.title('Employee Shift Schedule (Mixed Integer Programming)')
plt.axis('tight')
plt.yticks(np.arange(20), employees)
#modify ticks on y-axis
my_x_ticks = np.arange(0, 24, 1)      
plt.xticks(my_x_ticks)
plt.show()

In [ ]:
plt.xlabel('Time of day')
plt.ylabel('Numbers of employee')
plt.title('Hourly Staff Count(Mixed Integer Programming)')
plt.plot(actual_employee_working, '.')
plt.plot(demand_employee_working, '-')
plt.legend(['employees scheduled', 'employees required'])
plt.show()